# Chapter 5. Persistence

## 5.1 Introduction

### 5.1.1 Data persistence

(1) Storing bookmarks

(2) Storing user details

### 5.1.2 Models

(1) ORM: `Sqlalchemy`

(2) Model classes and database tables

(3) Simple relations and queries

(4) Database administration with `Flask-script`

## 5.2 Demo: Our first model classes

Install the Python packages `SQLAlchemy`, `flask-sqlalchemy`.

```bash
$ pip install flask-sqlalchemy
```

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from thermos import db

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    
    def __repr__(self):
        return '<User %r>' % self.username

In [ ]:
# SAVE AS thermos.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
from datetime import datetime

from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy 

from forms import BookmarkForm

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)

app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
db = SQLAlchemy(app)

bookmarks = []

def store_bookmark(url, description):
    bookmarks.append(dict(
        url = url,
        description = description,
        user = "reindert",
        date = datetime.utcnow()    
    ))
    
def new_bookmarks(num):
    return sorted(bookmarks, key=lambda bm: bm['date'], reverse=True)[:num]

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=new_bookmarks(5))
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        store_bookmark(url, description)
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  
  
if __name__ == '__main__':
    app.run(debug=True)

## 5.3 Review: Models with flask-sqlalchemy

### 5.3.1 Setting up flask-sqlalchemy

(1) `pip install flask-sqlalchemy`

(2) Import and configure

```python
from flask_sqlalchemy import SQLAlchemy

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
db = SQLAlchemy(app)
```

(3) SQLalchemy also supports: MySQL, MS SQL, PostgreSQL, Oracle, and more.

### 5.3.2 Model Classes

(1) A model class represents a database table.

Every instance represents a row in that table.

(2) Inherit from `db.Model`.

(3) Flask-SQLAlchemy -- sort of wrapper around `SQLAlchemy`

* Takes care of some things that plain `SQLAlchemy` does not.

* Don't have to set `__tablename__` for models.

* query attribute on models used for querying.

### 5.3.3 Columns

```python
id = db.Column(db.Integer, primary_key=True)
url = db.Column(db.Text, nullable=False)
description = db.Column(db.String(300))
```

(1) Define as class attributes on the Model class.

Name of database column will be name of the attribute.

(2) Instance of `db.Column`

* Data type: `db.Integer`, `db.String`, etc.

* Options: `primary_key`, `nullable`, `unique`, etc.

## 5.4 Demo: Storing and retrieving data

```bash
$ python
```

```python
Python 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from thermos import db
/home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/thermos/forms.py:5: ExtDeprecationWarning: Importing flask.ext.wtf is deprecated, use flask_wtf instead.
  from flask.ext.wtf.html5 import URLField
/home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/thermos/forms.py:5: FlaskWTFDeprecationWarning: "flask_wtf.html5" will be removed in 1.0.  Import directly from "wtforms.fields.html5" and "wtforms.widgets.html5".
  from flask.ext.wtf.html5 import URLField
/home/renwei/anaconda3/envs/web/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
>>> from models import User, Bookmark
>>> db.create_all()
>>> u = User(username="reindert", email="info@example.com")
>>> db.session.add(u)
>>> db.session.commit()
>>> from datetime import datetime
>>> db.session.add(Bookmark(url='http://www.pluralsight.com', date=datetime.utcnow(), description='Hardcore Developer Training'))
>>> db.session.commit()
>>> User.query
<flask_sqlalchemy.BaseQuery object at 0x7f63987bf390>
>>> User.query.get(1)
<User 'reindert'>
>>> User.query.get(2)
>>> User.query.filter_by(username='reindert')
<flask_sqlalchemy.BaseQuery object at 0x7f63982809b0>
>>> User.query.filter_by(username='reindert').all()
[<User 'reindert'>]
```

## 5.5 Review: Storing data and simple queries

### 5.5.1 Creating new data

(1) Simply create a new instance of the class

```python
bm = new Bookmark(user=u, url=url, description=desc)
```

Don't need to set the primary key `id` since it will be set automatically set.

(2) Add it to the database session

* `db.session.add(bm)`

* This does NOT add the data to the database.

* It registers the object with the session.

(3) Don't forget to `commit`.

* `db.session.commit()`

* Will run an `SQL INSERT` command.

### 5.5.2 Simple queries

(1) Using the `query` attribute of a model class

(2) Get by primary key

`Bookmark.query.get(1)`

(3) Retrieve all rows

`Bookmark.query.all()`

(4) Select specific rows

* `Bookmark.query.filter_by(username='reindert').first()`

* `Bookmark.query.filter_by(username='reindert').all()`

## 5.6 Demo: Storing and retrieving new bookmarks

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc

from thermos import db

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    
    def __repr__(self):
        return '<User %r>' % self.username

In [ ]:
# SAVE AS thermos.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
from datetime import datetime

from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy 

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)

app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
db = SQLAlchemy(app)

# Move the import statements after db to work around some circular-import issue.
from forms import BookmarkForm
import models

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=models.Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = models.Bookmark(url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  
  
if __name__ == '__main__':
    app.run(debug=True)

## 5.7 Demo: A manager script with flask-script

### 5.7.1 Install the Python package `flask-script`.

```bash
$ pip install flask-script
```

### 5.7.2 Create the manager script.

In [ ]:
# SAVE AS manager.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from thermos import app, db
from flask.ext.script import Manager, prompt_bool

manager = Manager(app)

@manager.command
def initdb():
    db.create_all()
    print('Initialized the database')
    
@manager.command
def dropdb():
    if prompt_bool(
        'Are you sure you want to lose all your data'):
        db.drop_all()
        print('Dropped the database')
        
if __name__ == '__main__':
    manager.run()

### 5.7.3 Manager commands

(1) Get the help information.

```bash
$ python manager.py 
usage: manager.py [-?] {initdb,dropdb,shell,runserver} ...

positional arguments:
  {initdb,dropdb,shell,runserver}
    initdb
    dropdb
    shell               Runs a Python shell inside Flask application context.
    runserver           Runs the Flask development server i.e. app.run()

optional arguments:
  -?, --help            show this help message and exit

```

(2) Run the Flask application.

```bash
$ python manager.py runserver
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```

(3) Drop all the data.

```bash
$ python manager.py dropdb
Are you sure you want to lose all your data [n]: y
Dropped the database
```

(4) Initialize the database.

```bash
$ python manager.py initdb
Initialized the database
```

## 5.8 Demo: A one-to-many relation

(1) Add a foreign key `user.id` in the table 'bookmark'.

(2) Create a relationship `bookmarks` in the table `user`.

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc

from thermos import db

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    
    def __repr__(self):
        return '<User %r>' % self.username

In [ ]:
# SAVE AS manager.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from thermos import app, db
from flask.ext.script import Manager, prompt_bool

from models import User

manager = Manager(app)

@manager.command
def initdb():
    db.create_all()
    db.session.add(User(username='reindert', email='reindert@example.com'))
    db.session.add(User(username='arjen', email='arjen@example.com'))
    db.session.commit()
    print('Initialized the database')
    
@manager.command
def dropdb():
    if prompt_bool(
        'Are you sure you want to lose all your data'):
        db.drop_all()
        print('Dropped the database')
        
if __name__ == '__main__':
    manager.run()

```bash
$ python manager.py dropdb
Are you sure you want to lose all your data [n]: y
Dropped the database
 
$ python manager.py initdb
Initialized the database
```

```bash
$ python
Python 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from thermos import db
>>> from models import User, Bookmark
>>> u = User.query.get(1)
>>> u
<User 'reindert'>
>>> for s in ['test', 'another one', 'some more']"
  File "<stdin>", line 1
    for s in ['test', 'another one', 'some more']"
                                                 ^
SyntaxError: EOL while scanning string literal
>>> for s in ['test', 'another one', 'some more']:
...     db.session.add(Bookmark(user=u, url=s))
... 
>>> db.session.commit()
>>> u.bookmarks
<sqlalchemy.orm.dynamic.AppenderBaseQuery object at 0x7f9d6a8f6630>
>>> u.bookmarks.all()
[<Bookmark 'None': 'test'>, <Bookmark 'None': 'another one'>, <Bookmark 'None': 'some more'>]
>>> u.bookmarks[1].user
<User 'reindert'>
```

In [ ]:
# SAVE AS thermos.py
# Note that the main function has been removed and we will solely use manager.py to run the server. 
# Also we need to modify index.html accordingly to display the user who adds the bookmark.

# -*- coding: utf-8 -*-

import os
from datetime import datetime

from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy 

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)

app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
db = SQLAlchemy(app)

from forms import BookmarkForm
import models
    
# Fake login
def logged_in_user():
    return models.User.query.filter_by(username='reindert').first()

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=models.Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = models.Bookmark(user=logged_in_user(), url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

## 5.9 Review: One-to-many relations and lazy loading

### 5.9.1 A one-to-many relation

```python
class Bookmark(db.Model):
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
class User(db.Model):
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
```

(1) `db.relationship`

* Defines a one-to-many relation.

* First argument gives **many** side of the relation

(2) `backref`

Name of an attribute on the related object

(3) `lazy`

How the related rows should be loaded

### 5.9.2 Lazy loading options

(1) `lazy = 'select'` (default)

Load the data lazily using a standard `SELECT` statement

(2) `lazy = 'joined'`

Load the data in the same query as the parent using a `JOIN` statement

(3) `lazy = 'subquery'`

Like 'joined' but use a subquery

(4) `lazy = 'dynamic'`

* Useful if you have many items.

* Returns a query object which you can further refine before loading items.

* Usually what you want if you expect more than a handful of items.

## 5.10 Breaking the project up into smaller files

Solve the circular-import issue.

In [ ]:
# SAVE AS __init__.py

# -*- coding: utf-8 -*-

import os

from flask import Flask
from flask_sqlalchemy import SQLAlchemy 

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
app.config['DEBUG'] = True

db = SQLAlchemy(app)

from . import models
from . import views

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc

from . import db

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    
    def __repr__(self):
        return '<User %r>' % self.username

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for

from . import app, db
from thermos.forms import BookmarkForm
from thermos.models import User, Bookmark 
    
# Fake login
def logged_in_user():
    return User.query.filter_by(username='reindert').first()

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = Bookmark(user=logged_in_user(), url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

In [ ]:
# SAVE AS manager.py
# Note that it needs to be moved one level up.

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from flask.ext.script import Manager, prompt_bool

from thermos import app, db
from thermos.models import User

manager = Manager(app)

@manager.command
def initdb():
    db.create_all()
    db.session.add(User(username='reindert', email='reindert@example.com'))
    db.session.add(User(username='arjen', email='arjen@example.com'))
    db.session.commit()
    print('Initialized the database')
    
@manager.command
def dropdb():
    if prompt_bool(
        'Are you sure you want to lose all your data'):
        db.drop_all()
        print('Dropped the database')
        
if __name__ == '__main__':
    manager.run()

## 5.11 Demo: Adding a user page

(1) Create a new template `user.html`.

(2) Specify the route to `user.html` in `views.py`.

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for

from . import app, db
from thermos.forms import BookmarkForm
from thermos.models import User, Bookmark 
    
# Fake login
def logged_in_user():
    return User.query.filter_by(username='reindert').first()

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = Bookmark(user=logged_in_user(), url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

## 5.12 Final review

### 5.12.1 Flask-SQLAlchemy helpers

```python
@app.route('/user/<username>')
def user(username):
```

(1) `first_or_404()`

(2) `get_or_404()`

### 5.12.2 Simple database administration

(1) `db.create_all()`

(2) `db.drop_all()`

(3) Use `Flask-Script` to create an admin interface.

## 5.13 Resources and summary

### 5.13.1 Resources

(1) `Flask-SQLAlchemy`

http://pythonhosted.org/Flask-SQLAlchemy/

(2) `SQLAlchemy`

http://docs.sqlalchemy.org/en/latest

(3) `Flask-Script`

http://flask-script.readthedocs.org/en/latest

### 5.13.2 Summary

(1) `Flask-SQLAlchemy`

* Models
* Columns
* Create and drop
* Insert data
* Simple queries
* One-to-many relations
* `first_or_404`

(2) `Flask-Script`

(3) Breaking up the application into multiple files.